# Applied Data Science Capstone Project

#### Gregory Smith

The body of this notebook consists  the Applied Data Science capstone project as part of the Applied Data Science specialization on Coursera.

In [1]:
import pandas as pd
import numpy as np

In [2]:
print('Hello Capstone Project Course!')

Hello Capstone Project Course!


## Segmenting and Clustering Neighborhoods in Toronto

### Importing and Cleaning Dataframe

Scraping Toronto postal codes from Wikipedia

In [219]:
import urllib.request

url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
req = urllib.request.urlopen(url)
article = req.read().decode()

with open('List_of_postal_codes_of_Canada:_M.html', 'w') as fo:
    fo.write(article)

df_toronto_neigh = pd.read_html('List_of_postal_codes_of_Canada:_M.html')[0]

In [26]:
df_toronto_neigh.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


Dropping postcodes that are unassigned to a borough

In [46]:
df_toronto_neigh = df_toronto_neigh[df_toronto_neigh['Borough']!='Not assigned']
df_toronto_neigh.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor


Creating a new dataframe which has all neighbourhoods under the same postal code and borough grouped together. This was done by creating a blank dataframe, creating temporary dataframes of postal code borough combinations, concatentating the neighbourhoods within this dataframe, and concatentating the resulting row with the new dataframe.

In [215]:
df_toronto_neigh_2 = pd.DataFrame(columns=['Postcode','Borough','Neighbourhood'])
for i, borough in enumerate(df_toronto_neigh['Borough'].unique()):
    for j, post in enumerate(df_toronto_neigh['Postcode'].unique()):
        temp_df = df_toronto_neigh.loc[(df_toronto_neigh['Borough']==borough) & (df_toronto_neigh['Postcode']==post)]
        while temp_df.shape[0]>1:
            temp_df.iloc[0,2]=temp_df.iloc[0,2]+', '+temp_df.iloc[1,2]
            temp_df.drop(temp_df.index[1], inplace=True)
        df_toronto_neigh_2=pd.concat([df_toronto_neigh_2,temp_df])

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [216]:
df_toronto_neigh_2.head()

,Postcode,Borough,Neighbourhood
10,M1B,Scarborough,"Rouge, Malvern"
26,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
41,M1E,Scarborough,"Guildwood, Morningside, West Hill"
52,M1G,Scarborough,Woburn
61,M1H,Scarborough,Cedarbrae


Finding which boroughs do not have an assigned neighborhood

In [232]:
df_toronto_neigh_2.loc[df_toronto_neigh_2['Neighbourhood']=='Not assigned']

,Postcode,Borough,Neighbourhood
9,M9A,Queen's Park,Not assigned


Assigning boroughs without an assigned neigborhood to have the same name as the borough

In [245]:
for i in range(df_toronto_neigh_2.shape[0]):
    if (df_toronto_neigh_2.iloc[i,2]=='Not assigned'):
        df_toronto_neigh_2.iloc[i,2] = df_toronto_neigh_2.iloc[i,1]

In [246]:
df_toronto_neigh_2.loc[df_toronto_neigh_2['Neighbourhood']=='Not assigned']

,Postcode,Borough,Neighbourhood


In [247]:
df_toronto_neigh_2.loc[df_toronto_neigh_2['Neighbourhood']==df_toronto_neigh_2['Borough']]

,Postcode,Borough,Neighbourhood
9,M9A,Queen's Park,Queen's Park


Cleaning dataframe by renaming a column and reseting the indices

In [250]:
df_toronto_neigh_2.rename(columns={'Postcode': 'PostalCode'}, inplace=True)

In [256]:
df_toronto_neigh_2.reset_index(drop=True, inplace=True)

Details of final dataframe 'df_toronto_neigh_2'

In [257]:
df_toronto_neigh_2.head()

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [262]:
df_toronto_neigh_2.describe()

,PostalCode,Borough,Neighbourhood
count,103,103,103
unique,103,11,102
top,M4V,North York,Queen's Park
freq,1,24,2


In [263]:
df_toronto_neigh_2.shape

(103, 3)